# 1. Params and loading packages

In [1]:
## First go back up a folder 
import os
os.chdir('..')

In [2]:
## Parameters specific to where your folders are and your data
parameter_file = 'params/params.yaml'
import yaml
import sys

with open(parameter_file,'r') as f:
    doc = yaml.load(f)

#p = dic2obj(**doc)

data_folder = doc['data_folder']
tissues = doc['tissues'].split(',')
sys.path.append(doc['pipeline_path'])
ref_fa = doc['ref_fa']
annotation=doc['annotation']



In [3]:

import os
import sys
import pandas as pd
import matplotlib
import seaborn as sns
import pickle
import numpy as np
import matplotlib as mpl
from matplotlib import pyplot as plt
from itertools import product
import glob
import re
from matplotlib_venn import venn2
from matplotlib import rcParams
import inspect
%matplotlib inline
%config InlineBackend.figure_format = 'retina'
sys.setrecursionlimit(3000)
%load_ext autoreload
%autoreload 2
rcParams['figure.figsize'] = 8, 6
import tqdm
### notebook specific configuration ###
from os.path import basename
#mpl.style.use('ggplot')
mpl.style.use('fivethirtyeight')
from cycler import cycler
mpl.rcParams['axes.prop_cycle'] = cycler(color='bgrcmyk')

from Modules.Homer import *

print('Number of tissues: ',len(tissues))

('Number of tissues: ', 16)


In [4]:
if not os.path.exists('Results'):
    os.mkdir('Results')
    
if not os.path.exists('Results/merged'):
    os.mkdir('Results/merged')

# Merge all tag directories instead of peak files

In [71]:
%%time
all_tags_start = []
for t in tissues:
    print(t)
    #curr = glob.glob(j + '*f03_tags/trim*.peak')
    curr_raw = glob.glob(data_folder + t + '/*/')
    #print(curr_raw)
    for j in curr_raw:
        curr_type = j.split('/')[-2]  
        if curr_type == 'GRO':
            #print(os.path.join(j,'f03_tags/*5GRO*'))
            new = glob.glob(os.path.join(j,'f03_tags/*5GRO*'))
        elif curr_type == 'mSTART':
            #print(os.path.join(j,'f03_tags/*5GRO*'))
            new = np.array(glob.glob(os.path.join(j,'f03_tags/*')))
            if not len(new) == 0:
                new = (new[(np.array(map(lambda x: 'input' not in x,new)))])
        else:
            continue
        all_tags_start.extend(new)
        


curr_out = 'Results/merged/tags_TSS_merged'
cmd = 'makeTagDirectory {out_f} -single -d {files}'.format(out_f=curr_out,files = ' '.join(all_tags_start))
print(cmd)
!{cmd}

BMDM1hKLA1
BMDMwt
Brain
FemaleReproductive
Heart
Intestine
Kidney
Liver
Lung
MiscOrgans
Muscle
Pancreas
Skin
Spleen
Total
CHO
makeTagDirectory Results/merged/tags_TSS_merged -single -d /data/isshamie/TSS/Processed/BMDMwt/GRO/f03_tags/trim_CHBMDMwt_5GRO_JHS1033_SD_TAGCTT_S48_L003_R1_001.fastq /data/isshamie/TSS/Processed/Brain/GRO/f03_tags/trim_CHBrain_5GRO_JHS1036_SD_ATGTCA_S51_L003_R1_001.fastq /data/isshamie/TSS/Processed/Brain/mSTART/f03_tags/trim_CHBrain_mSTART_JHS1016_SD_GTAGAG_S28_L002_R1_001.fastq /data/isshamie/TSS/Processed/Brain/mSTART/f03_tags/trim_CHBrain_mSTART_JHS1057_SD_ATTCCT_S79_L004_R1_001.fastq /data/isshamie/TSS/Processed/Heart/mSTART/f03_tags/trim_CHHeart_mSTART_JHS1013_SD_AGTTCC_S25_L002_R1_001.fastq /data/isshamie/TSS/Processed/Heart/mSTART/f03_tags/trim_CHHeart_mSTART_JHS1056_SD_ATGAGC_S78_L004_R1_001.fastq /data/isshamie/TSS/Processed/Kidney/GRO/f03_tags/trim_CHKidney_5GRO_JHS1035_SD_AGTTCC_S50_L003_R1_001.fastq /data/isshamie/TSS/Processed/Kidney/mSTART/f03_ta

	Adding tag directory /data/isshamie/TSS/Processed/Muscle/mSTART/f03_tags/trim_CHMuscle_mSTART_JHS1019_SD_GTGGCC_S31_L002_R1_001.fastq
	Adding tag directory /data/isshamie/TSS/Processed/Muscle/mSTART/f03_tags/trim_CHMuscle_mSTART_JHS1058_SD_CAAAAG_S80_L004_R1_001.fastq
	Adding tag directory /data/isshamie/TSS/Processed/Pancreas/mSTART/f03_tags/trim_CHPancreas_mSTART_JHS1014_SD_ATGTCA_S26_L002_R1_001.fastq
	Adding tag directory /data/isshamie/TSS/Processed/Pancreas/mSTART/f03_tags/trim_CHPancreas_mSTART_JHS1051_SD_GTTTCG_S73_L004_R1_001.fastq
	Adding tag directory /data/isshamie/TSS/Processed/Spleen/GRO/f03_tags/trim_CHBloondnegSpleen_5GRO_JHS1039_SD_GTCCGC_S54_L003_R1_001.fastq
	Adding tag directory /data/isshamie/TSS/Processed/Spleen/mSTART/f03_tags/trim_CHSpleen_mSTART_JHS1053_SD_GAGTGG_S75_L004_R1_001.fastq
	Adding tag directory /data/isshamie/TSS/Processed/Spleen/mSTART/f03_tags/trim_CHSpleednegBlood_mSTART_JHS1017_SD_GTCCGC_S29_L002_R1_001.fastq
	Adding tag directory /data/isshami

In [75]:
%%time
all_tags_input = []
for t in tissues:
    print(t)
    #curr = glob.glob(j + '*f03_tags/trim*.peak')
    curr_raw = glob.glob(data_folder + t + '/*/')
    #print(curr_raw)
    for j in curr_raw:
        curr_type = j.split('/')[-2]  
        if curr_type == 'GRO':
            #print(os.path.join(j,'f03_tags/*5GRO*'))
            new = glob.glob(os.path.join(j,'f03_tags/*_GRO*'))
            new.extend(glob.glob(os.path.join(j,'f03_tags/*-GRO*')))
        elif curr_type == 'mSTART':
            new = np.array(glob.glob(os.path.join(j,'f03_tags/*input*')))

        else:
            continue
        all_tags_input.extend(new)
        


curr_out = 'Results/merged/tags_input_merged'
cmd = 'makeTagDirectory {out_f} -single -d {files}'.format(out_f=curr_out,files = ' '.join(all_tags_input))
print(cmd)
!{cmd}

BMDM1hKLA1
BMDMwt
Brain
FemaleReproductive
Heart
Intestine
Kidney
Liver
Lung
MiscOrgans
Muscle
Pancreas
Skin
Spleen
Total
CHO
makeTagDirectory Results/merged/tags_input_merged -single -d /data/isshamie/TSS/Processed/BMDMwt/GRO/f03_tags/trim_CHBMDMwt_GRO_JHS1026_SD_ATTCCT_S41_L003_R1_001.fastq /data/isshamie/TSS/Processed/Brain/GRO/f03_tags/trim_CHBrain_GRO_JHS1029_SD_CACCGG_S44_L003_R1_001.fastq /data/isshamie/TSS/Processed/Brain/mSTART/f03_tags/trim_CHBrain_mSTARTinput_JHS1078_SD_ATTCCT_S100_L005_R1_001.fastq /data/isshamie/TSS/Processed/Heart/mSTART/f03_tags/trim_CHHeart_mSTARTinput_JHS1077_SD_ATGAGC_S99_L005_R1_001.fastq /data/isshamie/TSS/Processed/Kidney/GRO/f03_tags/trim_CHKidney_GRO_JHS1028_SD_CAACTA_S43_L003_R1_001.fastq /data/isshamie/TSS/Processed/Kidney/mSTART/f03_tags/trim_CHKidney_mSTARTinput_JHS1080_SD_CAACTA_S102_L005_R1_001.fastq /data/isshamie/TSS/Processed/Liver/GRO/f03_tags/trim_CHLiver_GRO_JHS1031_SD_TAATCG_S46_L003_R1_001.fastq /data/isshamie/TSS/Processed/Liver/mS

## Find peaks and annotate

In [87]:
%%time
tag_dir = 'Results/merged/tags_TSS_merged'
control_dir = 'Results/merged/tags_input_merged'
peak_style = 'tss'
out_file = 'Results/merged/tags_merged.peak'
bg = '-F 2'
otherParams = [bg]

cmd = "findPeaks {tag} -style {style} -o {out} -i {control} -norm 1000000 {other}".format(
                tag=tag_dir,style=peak_style,out=out_file,control=control_dir,
                other=' '.join(otherParams))
print(cmd)
!{cmd}


findPeaks Results/merged/tags_TSS_merged -style tss -o Results/merged/tags_merged.peak -i Results/merged/tags_input_merged -norm 1000000 -F 2
	Fragment Length = 1
	Finding tags on separate strands: doubling effective genome size
	Finding peaks of size 150, no closer than 300
		Finding peaks on picr_0 (minCount=9.9), total tags positions = 1833029
		Finding peaks on picr_1 (minCount=9.9), total tags positions = 1187057
		Finding peaks on picr_10 (minCount=9.9), total tags positions = 638255
		Finding peaks on picr_100 (minCount=9.9), total tags positions = 145711
		Finding peaks on picr_1000 (minCount=9.9), total tags positions = 10
		Finding peaks on picr_1001 (minCount=9.9), total tags positions = 4
		Finding peaks on picr_1002 (minCount=9.9), total tags positions = 10
		Finding peaks on picr_1003 (minCount=9.9), total tags positions = 4
		Finding peaks on picr_1005 (minCount=9.9), total tags positions = 11
		Finding peaks on picr_1006 (minCount=9.9), total tags positions = 134
		Find

		Finding peaks on picr_1110 (minCount=9.9), total tags positions = 6
		Finding peaks on picr_1111 (minCount=9.9), total tags positions = 2
		Finding peaks on picr_1112 (minCount=9.9), total tags positions = 183
		Finding peaks on picr_1113 (minCount=9.9), total tags positions = 2
		Finding peaks on picr_1114 (minCount=9.9), total tags positions = 6
		Finding peaks on picr_1115 (minCount=9.9), total tags positions = 30
		Finding peaks on picr_1116 (minCount=9.9), total tags positions = 47
		Finding peaks on picr_1118 (minCount=9.9), total tags positions = 26
		Finding peaks on picr_1119 (minCount=9.9), total tags positions = 7
		Finding peaks on picr_112 (minCount=9.9), total tags positions = 75489
		Finding peaks on picr_1120 (minCount=9.9), total tags positions = 5
		Finding peaks on picr_1121 (minCount=9.9), total tags positions = 1
		Finding peaks on picr_1123 (minCount=9.9), total tags positions = 214
		Finding peaks on picr_1124 (minCount=9.9), total tags positions = 2
		Finding 

		Finding peaks on picr_1230 (minCount=9.9), total tags positions = 3
		Finding peaks on picr_1231 (minCount=9.9), total tags positions = 9
		Finding peaks on picr_1232 (minCount=9.9), total tags positions = 8
		Finding peaks on picr_1233 (minCount=9.9), total tags positions = 4
		Finding peaks on picr_1234 (minCount=9.9), total tags positions = 2
		Finding peaks on picr_1237 (minCount=9.9), total tags positions = 4
		Finding peaks on picr_1238 (minCount=9.9), total tags positions = 4
		Finding peaks on picr_1239 (minCount=9.9), total tags positions = 37
		Finding peaks on picr_124 (minCount=9.9), total tags positions = 84838
		Finding peaks on picr_1240 (minCount=9.9), total tags positions = 101
		Finding peaks on picr_1241 (minCount=9.9), total tags positions = 4
		Finding peaks on picr_1242 (minCount=9.9), total tags positions = 1
		Finding peaks on picr_1243 (minCount=9.9), total tags positions = 4
		Finding peaks on picr_1244 (minCount=9.9), total tags positions = 1
		Finding peak

		Finding peaks on picr_1350 (minCount=9.9), total tags positions = 20
		Finding peaks on picr_1351 (minCount=9.9), total tags positions = 2
		Finding peaks on picr_1352 (minCount=9.9), total tags positions = 2
		Finding peaks on picr_1353 (minCount=9.9), total tags positions = 3
		Finding peaks on picr_1354 (minCount=9.9), total tags positions = 1
		Finding peaks on picr_1355 (minCount=9.9), total tags positions = 4
		Finding peaks on picr_1356 (minCount=9.9), total tags positions = 3
		Finding peaks on picr_1357 (minCount=9.9), total tags positions = 279
		Finding peaks on picr_1358 (minCount=9.9), total tags positions = 2
		Finding peaks on picr_136 (minCount=9.9), total tags positions = 108847
		Finding peaks on picr_1360 (minCount=9.9), total tags positions = 9
		Finding peaks on picr_1363 (minCount=9.9), total tags positions = 8
		Finding peaks on picr_1364 (minCount=9.9), total tags positions = 6
		Finding peaks on picr_1366 (minCount=9.9), total tags positions = 9
		Finding pea

		Finding peaks on picr_1480 (minCount=9.9), total tags positions = 6
		Finding peaks on picr_1481 (minCount=9.9), total tags positions = 3
		Finding peaks on picr_1482 (minCount=9.9), total tags positions = 1
		Finding peaks on picr_1483 (minCount=9.9), total tags positions = 3
		Finding peaks on picr_1484 (minCount=9.9), total tags positions = 1
		Finding peaks on picr_1485 (minCount=9.9), total tags positions = 7
		Finding peaks on picr_1486 (minCount=9.9), total tags positions = 12
		Finding peaks on picr_1487 (minCount=9.9), total tags positions = 7
		Finding peaks on picr_1488 (minCount=9.9), total tags positions = 1
		Finding peaks on picr_1489 (minCount=9.9), total tags positions = 10
		Finding peaks on picr_149 (minCount=9.9), total tags positions = 20557
		Finding peaks on picr_1490 (minCount=9.9), total tags positions = 823
		Finding peaks on picr_1491 (minCount=9.9), total tags positions = 5
		Finding peaks on picr_1492 (minCount=9.9), total tags positions = 4
		Finding pea

...

		Finding peaks on picr_490 (minCount=9.9), total tags positions = 22
		Finding peaks on picr_491 (minCount=9.9), total tags positions = 46
		Finding peaks on picr_492 (minCount=9.9), total tags positions = 93
		Finding peaks on picr_493 (minCount=9.9), total tags positions = 27
		Finding peaks on picr_494 (minCount=9.9), total tags positions = 14
		Finding peaks on picr_495 (minCount=9.9), total tags positions = 5
		Finding peaks on picr_496 (minCount=9.9), total tags positions = 34
		Finding peaks on picr_497 (minCount=9.9), total tags positions = 8
		Finding peaks on picr_498 (minCount=9.9), total tags positions = 66
		Finding peaks on picr_499 (minCount=9.9), total tags positions = 5
		Finding peaks on picr_5 (minCount=9.9), total tags positions = 801141
		Finding peaks on picr_50 (minCount=9.9), total tags positions = 440534
		Finding peaks on picr_500 (minCount=9.9), total tags positions = 15
		Finding peaks on picr_501 (minCount=9.9), total tags positions = 3
		Finding peaks on 

		Finding peaks on picr_60 (minCount=9.9), total tags positions = 82313
		Finding peaks on picr_600 (minCount=9.9), total tags positions = 16
		Finding peaks on picr_601 (minCount=9.9), total tags positions = 16
		Finding peaks on picr_602 (minCount=9.9), total tags positions = 13
		Finding peaks on picr_603 (minCount=9.9), total tags positions = 269
		Finding peaks on picr_604 (minCount=9.9), total tags positions = 11
		Finding peaks on picr_605 (minCount=9.9), total tags positions = 329
		Finding peaks on picr_606 (minCount=9.9), total tags positions = 336
		Finding peaks on picr_607 (minCount=9.9), total tags positions = 26
		Finding peaks on picr_608 (minCount=9.9), total tags positions = 44
		Finding peaks on picr_609 (minCount=9.9), total tags positions = 2
		Finding peaks on picr_61 (minCount=9.9), total tags positions = 422907
		Finding peaks on picr_610 (minCount=9.9), total tags positions = 6
		Finding peaks on picr_611 (minCount=9.9), total tags positions = 4
		Finding peaks

		Finding peaks on picr_710 (minCount=9.9), total tags positions = 20
		Finding peaks on picr_711 (minCount=9.9), total tags positions = 16
		Finding peaks on picr_712 (minCount=9.9), total tags positions = 9
		Finding peaks on picr_713 (minCount=9.9), total tags positions = 40
		Finding peaks on picr_714 (minCount=9.9), total tags positions = 2
		Finding peaks on picr_715 (minCount=9.9), total tags positions = 28
		Finding peaks on picr_716 (minCount=9.9), total tags positions = 4
		Finding peaks on picr_717 (minCount=9.9), total tags positions = 3
		Finding peaks on picr_718 (minCount=9.9), total tags positions = 13
		Finding peaks on picr_719 (minCount=9.9), total tags positions = 26
		Finding peaks on picr_72 (minCount=9.9), total tags positions = 115514
		Finding peaks on picr_720 (minCount=9.9), total tags positions = 29
		Finding peaks on picr_721 (minCount=9.9), total tags positions = 158
		Finding peaks on picr_722 (minCount=9.9), total tags positions = 47
		Finding peaks on p

		Finding peaks on picr_820 (minCount=9.9), total tags positions = 3
		Finding peaks on picr_821 (minCount=9.9), total tags positions = 87
		Finding peaks on picr_822 (minCount=9.9), total tags positions = 12
		Finding peaks on picr_823 (minCount=9.9), total tags positions = 6
		Finding peaks on picr_824 (minCount=9.9), total tags positions = 65
		Finding peaks on picr_825 (minCount=9.9), total tags positions = 2
		Finding peaks on picr_826 (minCount=9.9), total tags positions = 2
		Finding peaks on picr_827 (minCount=9.9), total tags positions = 11
		Finding peaks on picr_828 (minCount=9.9), total tags positions = 10
		Finding peaks on picr_829 (minCount=9.9), total tags positions = 248
		Finding peaks on picr_83 (minCount=9.9), total tags positions = 220483
		Finding peaks on picr_830 (minCount=9.9), total tags positions = 8
		Finding peaks on picr_831 (minCount=9.9), total tags positions = 15
		Finding peaks on picr_832 (minCount=9.9), total tags positions = 4
		Finding peaks on pic

		Finding peaks on picr_940 (minCount=9.9), total tags positions = 43
		Finding peaks on picr_941 (minCount=9.9), total tags positions = 5
		Finding peaks on picr_942 (minCount=9.9), total tags positions = 17
		Finding peaks on picr_943 (minCount=9.9), total tags positions = 17
		Finding peaks on picr_944 (minCount=9.9), total tags positions = 11
		Finding peaks on picr_945 (minCount=9.9), total tags positions = 2
		Finding peaks on picr_946 (minCount=9.9), total tags positions = 11
		Finding peaks on picr_947 (minCount=9.9), total tags positions = 34
		Finding peaks on picr_948 (minCount=9.9), total tags positions = 7
		Finding peaks on picr_95 (minCount=9.9), total tags positions = 87246
		Finding peaks on picr_950 (minCount=9.9), total tags positions = 9
		Finding peaks on picr_951 (minCount=9.9), total tags positions = 2
		Finding peaks on picr_952 (minCount=9.9), total tags positions = 5
		Finding peaks on picr_953 (minCount=9.9), total tags positions = 303
		Finding peaks on picr

In [94]:
%%time
output_anno_file = 'Results/merged/tags_merged.anno' 
peak_file = 'Results/merged/tags_merged.peak'


annotate_peaks(peak_file=peak_file,output_file=output_anno_file ,ref_fa=ref_fa,annotation=annotation)

annotatePeaks.pl Results/merged/tags_merged.peak /data/isshamie/genome/picr.fa -gff /data/isshamie/genome/updated_final_sort.gff3 > Results/merged/tags_merged.anno
